In [1]:
import re


# Assignment 2: Analyzing large datasets with Spark.


For this assignment, you will need to make sure you're running from a PySpark docker environment I introduced in class. You can start the docker pySpark docker environment using the following command:

```
docker run --rm -p 4040:4040 -p 8888:8888 -v $(pwd):/home/jovyan/work jupyter/all-spark-notebook
```

Make sure you run the command from the directory containing this jupyter notebook and your data folder.


</b>
# WARNING: For some reason, ipynbb document didn't always sync properly when I was pushing to github. As such, please push often and make sure your incremental changes appear on GitHub.
</b>

### Part 1

The first part will use Spark to analyze the following books, which I have downloaded for you to use from Project Gutenberg. The files are saved to the data folder.

| File name | Book Title|
|:---------:|:----------|
|43.txt | The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson|
|84.txt | Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley |
|398.txt  | The First Book of Adam and Eve by Rutherford Hayes Platt|
|3296.txt | The Confessions of St. Augustine by Bishop of Hippo Saint Augustine|

The objective is to explore whether we can detect similarity between books within the same topic using word-based similarity. 

The task of identifying similar texts in Natural Language Processing is crucial. A naive method for determining whether two documents are similar is to treat them as collections of words (bag of words) and use the number of words they share as a proxy for their similarity. It makes sense that two books with religion as the topic (e.g.  `398.txt` and `3296.txt`) would have more words in common than a book that discusses religion and a book that discusses science fiction (e.g. books `84.txt` and `398.txt`). 

As mentioned above, we will be using Spark to analyze the data. Although Spark is not needed for such a small example, the platform would be ideal for analyzing very large collections of documents, like those often analyzed by large corporations

This part of the assignment will rely exclusively on RDDs.

### Q1. 
Start by importing Spark and making sure your environment is set up properly for the assignment.

Import the spark context necessary to load a document as an RDD; ignore any error messages

In [2]:
import pyspark
sc = pyspark.SparkContext()

21/10/20 17:34:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Q2 

Read in the file `43.txt` as a spark RDD and save it to a variable called `book_43`
 * make sure `book_43` is of type MapPartitionsRDD, i.e.,
   * str(type(book_43)) == "<class 'pyspark.rdd.RDD'>" should return True 

In [4]:
book_43 = sc.textFile("data/43.txt")

assert str(type(book_43)) == "<class 'pyspark.rdd.RDD'>"

### Q3

How many lines does `book_43` contain?
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [9]:
lines_count = book_43.count()
print(f"book_43 has {lines_count} lines.")

book_43 has 2935 lines.


### Q4 

Prior to analyzing the words contained in this book, we need to first remove the occurrences of non-alphabetical characters and numbers from the text. You can use the following function, which takes a line as input, removes digits and non-word characters, and splits it into a collection of words. 

```python
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
```

Use the fucntion above on the variable (test_line) to see what it returns.
```python
test_line = "This is an example of that contains 234 and a dash-containing number"
```

In [13]:
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()

test_line = "This is an example of that contains 234 and a dash-containing number"
clean_split_line(test_line)

['THIS',
 'IS',
 'AN',
 'EXAMPLE',
 'OF',
 'THAT',
 'CONTAINS',
 'AND',
 'A',
 'DASH',
 'CONTAINING',
 'NUMBER']

### Q5

How many words does `book_43` contain? To answer this question, you may find it useful to apply the function in a spark-fashion. 
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [25]:
book_43_clean = book_43.map(clean_split_line)
lines_clean = book_43_clean.collect()
word_count = 0
for line in lines_clean:
    for word in line:
        word_count += 1
print(f"book_43 has {word_count} words.")

29116


### Q6

How many of the words in book_43 are unique? Given that words can appear in lower, upper or mixed case (ex. The, THE, the), make sure you convert the words into lower case before counting them.


In [32]:
unique_words = {}
for line in lines_clean:
    for word in line:
        if word.lower() not in unique_words:
            unique_words[word.lower()] = 1
        else:
            unique_words[word.lower()] += 1
print(f"book_43 has {len(unique_words)} unique words.")

book_43 has 4296 unique words.


### Q7

* Generate an `RDD` that contains the frequency of each word in `book_43`. Call the variable `book_43_counts`. Each item in the `RDD` should be a tuple with the word as the first element of the tuple and the count as the second item of the tuple. The collection should look like the following:

[('project', 88), ("the", 1807), ... ]

* Such a collection may contain a large number of words and it would be imprudent to transfer all the words onto the same machine to display it. Instead, to explore the content of such a collection, display only the first element in your list. 

* Given the random nature of this operation, the first element element displayed may be different. The first entry for me was:
```
[('project', 88)]
```

* You can only use operations or actions to answer the question. 
* Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
* Code that uses functions such as `some_func(...)` is not allowed


In [49]:
book_43_counts = sc.parallelize(list(unique_words.items()))
print(book_43_counts.collect()[:5])

[('the', 1807), ('project', 88), ('gutenberg', 98), ('ebook', 13), ('of', 1068)]


### Q8

Sort `book_43_counts` and print the 20 most frequent words in book_43. 
  * Hint: function `sortByKey` sorts a collection of tuples on the first element element of the list. You can easily change the order of the items in each element and use `sortByKey` to sort on the second item of each element in `book_43_counts`
  * You can only use operations or actions to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [64]:
book_43_counts_swapped = book_43_counts.map(lambda x: (x[1], x[0]))
book_43_counts_swapped.sortByKey(ascending=False).collect()[:20]

[(1807, 'the'),
 (1068, 'of'),
 (1043, 'and'),
 (726, 'to'),
 (686, 'a'),
 (646, 'i'),
 (485, 'in'),
 (471, 'was'),
 (392, 'that'),
 (384, 'he'),
 (378, 'it'),
 (312, 'you'),
 (308, 'my'),
 (301, 'with'),
 (285, 'his'),
 (244, 'had'),
 (203, 'as'),
 (202, 'for'),
 (195, 'this'),
 (193, 'but')]

### Q9

You must have noted that the most frequent words in `book_43_counts` include stop words such as `of`, `the`, `and`, etc.

It would be inefficient to compare documents based on whether or not they contain stop words; those are common to all documents. As such, it's common to remove such stop words. The librarary `sklearn.feature_extraction` provides access to a collection of English stop words, which can be loaded using the following snippet:

```
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS
```

* Explore ENGLISH_STOP_WORDS (it's a frozen set data structure, i.e., a set that you cannot modify) by printing any 10 words from it. 
 * Hint: convert the frozen set to something you can subscript


In [68]:
from sklearn.feature_extraction import _stop_words
stop_words_list = list(_stop_words.ENGLISH_STOP_WORDS)
stop_words_list[:10]

['that',
 'although',
 'nowhere',
 'for',
 'might',
 'never',
 'above',
 'con',
 'please',
 'much']

### Q10

Filter out the words in `book_43_counts` by removing those that appear in the ENGLISH_STOP_WORDS.
Save the results to a new variable called `book_43_counts_filtered`
  * You can only use operarations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses function such as `some_func(...)` is not allowed


In [79]:
book_43_counts_filtered = book_43_counts.filter(lambda x: x[0] not in stop_words_list)

### Q11

* How many words are left in `book_43_counts_filtered` after removing the stop words

In [78]:
print(f"Reduced {book_43_counts.count()} words to {book_43_counts_filtered.count()}.")

Reduced 4296 words to 4034.


### Q12 

* Create a function called *process_RDD* that combines the relevant steps you proposed above to make it convenient to apply them to the remaining four books. Your function should accept an input text file path and:
 * Reads in the file as a textRDD
 * Cleans and splits the line using `clean_split_line`
 * Filters out the stop words
 * Returns a word count RDD where each item is a tuple of words and its count.
 


In [93]:
def process_RDD(path: str):
    rdd = sc.textFile(path)
    rdd_clean = rdd.map(clean_split_line)
    lines_clean = rdd_clean.collect()
    
    unique_words = {}
    for line in lines_clean:
        for word in line:
            if word.lower() not in unique_words:
                unique_words[word.lower()] = 1
            else:
                unique_words[word.lower()] += 1
                
    rdd_counts = sc.parallelize(list(unique_words.items()))
    rdd_counts_filtered = rdd_counts.filter(lambda x: x[0] not in stop_words_list)
    return rdd_counts_filtered



### Q13 

Apply the function `process_RDD` to `book_84`, `book_398` and `book_3296` and save the results to variables `book_84_counts_filtered`, `book_398_counts_filtered` and `book_3296_counts_filtered` respectively. How many distinct words does each book contain after filtering the stop words.


In [98]:
book_84_counts_filtered = process_RDD("data/84.txt")
book_398_counts_filtered = process_RDD("data/398.txt")
book_3296_counts_filtered = process_RDD("data/3296.txt")

print(f"book_84 has {book_84_counts_filtered.count()} distinct words.")
print(f"book_398 has {book_398_counts_filtered.count()} distinct words.")
print(f"book_3296 has {book_3296_counts_filtered.count()} distinct words.")


book_84 has 7016 distinct words.
book_398 has 2421 distinct words.
book_3296 has 7293 distinct words.


### Q14 

We discussed how to evaluate similarity between two texts using the number of words they share. We hypothesized that books that are similar should have more words in common than books that are dissimilar. If that holds, `book_398` and `book_3296`, which both pertain to religion, will have more words in common than, say, `book_84` and `book_398`. Test this hypothesis by writing code that compares and prints the number of words shared between `book_398` and `book_3296` and then between `book_84` and `book_398`.


In [118]:
def compareWords(path_1: str, path_2:str):
    book_1 = process_RDD(path_1)
    book_2 = process_RDD(path_2)
    common_words = book_1.join(book_2).collect()
    return len(common_words)

common_398_3296 = compareWords("data/398.txt", "data/3296.txt")
common_398_84 = compareWords("data/398.txt", "data/84.txt")

print(f"book_398 and book_3296 have {common_398_3296} words in common.")
print(f"book_398 and book_84 have {common_398_84} words in common.")

book_398 and book_3296 have 1790 words in common.
book_398 and book_84 have 1691 words in common.
book_3296 and book_84 have 3608 words in common.


### Q15

* Based on the above, do you think counting the number of shared words is a good idea as a distance metric for evaluating topic similarity? Justify your answer?
* Hint: What do *book_84* and *book_3296* have in common? 

Since both comparisons have a similar amount of common words but are dissimilar in content,
I conclude that this is not a useful metric for topic similarity.

Both book_84 and book_3296 have a similar amount of total words, which will just increase the probability to have common words with book_398.

The majority of common words probably don't consist of topic-related words but rather ubiqutious verbs or adjectives, prepositions etc:. he, she, will, go, below, above ...

## Part II 

Another approach to estimating similarity consists of computing the Euclidean distance across a set of words. For example, suppose we have 3 documents A, B and C with the following counts for the words `evolution`, `DNA`, `biology` and `finance`. 

```python 
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
```
Although all documents contain exactly the four words, the number of times these words appear in each book may be indicative of thier topic. For example, documents `A` and `B` are more likely to be business related since they contain the word `finance` more frequently (8 and 10 times respectively). Document `C` may be a technical document since it focuses on more technical words (`evolution` and `DNA`) and less on the words `finance`.

The Euclidean distance, which can be computed using the `scikit` snippet below, is more indicative of topic-relatedness between the two documents.

```python
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")
```


### Q16

To calculate the Euclidean distance, we must first identify the set of words by which we will compare the documents. Here, we will explore the words that are common to all 4 documents. We will store the data in a matrix called `counts_matrix`.

Start by finding the words that are common to all four documents after stop-word filtering and store the counts for each word in a column of `counts_matrix`. 

To take the previous example, you can generate an emtpy matrix with 3 lines (documents `A`, `B` and `C`) and 4 columns (words `evolution`, `DNA`, `biology` and `finance`) using the following code.

```python
import numpy as np
counts_matrix = np.zeros([3,4])
```

After generting the counts, you can fill the counts for a document, say `A`, using the following code:

```python
counts_matrix[0, :] = [4, 9, 6, 8] 
```
* Other than for building `counts_matrix` you should exclusively use operations or actions on the `RDD` to answer this question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [166]:
import numpy as np
counts_matrix = np.zeros([3,4])

book_43_processed = process_RDD("data/43.txt")
book_84_processed = process_RDD("data/84.txt")
book_398_processed = process_RDD("data/398.txt")
book_3296_processed = process_RDD("data/3296.txt")

# join all rdds and flatten the values so they are 4-tuples with word counts for each book:
# (book_43, book_84, book_398, book_3296)

all_common = book_43_processed.join(book_84_processed)
all_common = all_common.join(book_398_processed).mapValues(lambda x: x[0] + (x[1], ))
all_common = all_common.join(book_3296_processed).mapValues(lambda x: x[0] + (x[1], ))
print(all_common.keys().collect())
# convert directly to numpy matrix, now each row represents a unique word
# and each column reflects its count in the 4 books
all_common_matrix = np.array(all_common.values().collect())

# if we transpose it, we have the required matrix with counts_matrix[0, :]
# being a list of counts for all unique words in the first book
print(counts_matrix)
book_43_counts = counts_matrix[0, :]
book_84_counts = counts_matrix[1, :]
book_398_counts = counts_matrix[2, :]
book_3296_counts = counts_matrix[3, :]

['away', 's', 'walk', 'head', 'master', 'mean', 'water', 'distress', 'owns', 'entity', 'owner', 'transcription', 'additions', 'maintaining', 'harm', 'blew', 'cast', 'shattered', 'individual', 'appears', 'identify', 'obsolete', 'ein', 'check', 'set', 'money', 'matter', 'eyes', 'behold', 'tell', 'speech', 'different', 'trouble', 'offered', 'wonder', 'work', 'active', 'commercial', 'freely', 'breach', 'disclaimers', 'network', 'marked', 'sighed', 'lover', 'dust', 'nearly', 'trees', 'mountain', 'indicating', 'variety', 'non', 'terms', 'accept', 'clearly', 'meant', 'makes', 'paper', 'manner', 'city', 'special', 'calculated', 'main', 'solid', 'section', 'preserve', 'old', 'forth', 'sigh', 'returning', 'blow', 'greater', 'speaking', 'beginning', 'promising', 'imposed', 'specified', 'periodic', 'uniform', 'project', 'strange', 'sat', 'shadow', 'hurt', 'covered', 'sad', 'suffer', 'forced', 'secret', 'discover', 'language', 'ready', 'child', 'decided', 'beheld', 'grant', 'account', 'array', 'pos

### Q17

Compute the Euclidean distance between `book_398` and `book_3296`, which both talk about religion and `book_84` and `book_398`. What do you conclude about using the Euclidean distance for evaluating topic relatedness across documents?

In [164]:
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between book_398 and book_3296 is: {euclidean(book_398_counts, book_3296_counts)}")
print(f"The Euclidean distance between book_398 and book_84 is: {euclidean(book_398_counts, book_84_counts)}")

The Euclidean distance between book_398 and book_3296 is: 1156.6628722320086
The Euclidean distance between book_398 and book_84 is: 751.6688100486809


The distance between 398 and 3296 is still larger than between 398 and 84 despite their similarity in topic (religion).

Once again a more impactful factor is the similarity in total word counts, since
the majority of common words (in all 4 docs) are not tied to a topic (religion or science fiction).

Due to that, the euclidean distance between two books may solely be large because one book uses their common words way more frequently than the other.

### Q18

Bonus question (5 points): Can you think of a few things we could do to improve similarity between documents that pertain to the same topic. Justify your answer without giving code

One idea could be to normalize the frequency of word counts to the total word count,
i.e. that the matrix contains the relative, not the absolute word count.


So, for example, if 'prayer', 'faith', 'god' each make up 1% of two religion-related books, their individual total word count would not matter. Rather it's their percentual composition of unique words which would lead to a smaller euclidean distance.

## Part III

In this part we will build some basic analytics for a dataset consisting of flight arrival and departure details for all commercial flights within the USA in one month. While this dataset can be managed using Pandas (<1M records), scaling to a yearly or longer timeframe will greatly benefit from using a distributed computing framework such as `Spark`.

Here, you should use exclusively `SparkDatFrames. 

We want to analyze this dataset to better schedule trips.  For example:
 * Avoid airlines carriers that are most often associated with delays.
 * Avoid departure days where delays are most frequent.
 * Avoid airports which are associated with delays or long taxxying time.
* etc.
 

The information about the fields contained in the data file can be found [here](https://dataverse.harvard.edu/dataset.xhtml;jsessionid=0414e25969eccd0e88ae4d64fa0b?persistentId=doi%3A10.7910%2FDVN%2FHG7NV7&version=&q=&fileTypeGroupFacet=&fileTag=%221.+Documentation%22&fileSortField=date&fileSortOrder=desc)


### Q19

Load the file `flight_info.csv` into a spark `DataFrame` called `fight_info`.

  * Note that you will need to create a sparkSession prior to loading the data
  
* How many entries does the file contain?



In [272]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("assignment2") \
    .getOrCreate()

flight_info_df = spark.read.csv("data/flight_info.csv", header=True)
print(f"The file contains {flight_info_df.count()} entries.")

The file contains 450017 entries.


### Q20

Use `pySpark-SQL` or `pandas-like syntax to compute the airlines represented in this dataset
The airline information is stored in a field called UniqueCarrier
* UniqueCarrier: Represents the unique carrier code (ex.AA = American Airlines) 


In [242]:
flight_info_df.select("UniqueCarrier").distinct().show()

+-------------+
|UniqueCarrier|
+-------------+
|           UA|
|           NK|
|           AA|
|           EV|
|           B6|
|           DL|
|           OO|
|           F9|
|           HA|
|           AS|
|           VX|
|           WN|
+-------------+



### Q21

The data file contains various other fields, two of which are useful for answering the next question.

* CRSDepTime: Represents the scheduled departure time
* DepTime: Represents the actual departure time

Compute the number of flights delayed per each carried code represented in this dataset. Sort the data by decreasing order of delays.
  * A delay is observed when `DepTime` > `CRSDepTime`


In [273]:
flight_info_df.where(flight_info_df["DepTime"] > flight_info_df["CRSDepTime"]) \
    .groupBy('UniqueCarrier') \
    .count().orderBy('count', ascending=False).show()

+-------------+-----+
|UniqueCarrier|count|
+-------------+-----+
|           WN|54557|
|           DL|28962|
|           AA|26291|
|           UA|19594|
|           OO|17924|
|           EV|12340|
|           B6|10406|
|           AS| 4966|
|           NK| 4435|
|           F9| 3181|
|           VX| 2871|
|           HA| 2265|
+-------------+-----+



### Q22

 Use the file `airlines.csv` to find the the complete name of the airline. Here, you are required to load the file as a pyspark DataFrame; call it `airlines_info`, and repeat the query above while including the `flights.csv `file in your query ( requires doing a `join`) so that you can also display the full name of the carrier (second column). 

The result will look (approximately) like:

```
[Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count=SOME_count),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count=SOME_count),
 Row(UniqueCarrier='AA', first(_c1)='American Airlines', count=SOME_count),
 ...
 ]
```

The carrier code in the `airlines.csv` file is provided in the 4th (1-based) column

Note that the file `airlines.csv` does not have column header. Hence, you need to print one line of your dataset to see what names Spark gave to the columns. Use the name provided by Spark in your query.

In [303]:
airlines_info = spark.read.csv("data/airlines.csv", header=False)

flight_info_df.join(airlines_info, flight_info_df.UniqueCarrier == airlines_info._c3, 'inner') \
    .where(flight_info_df["DepTime"] > flight_info_df["CRSDepTime"]) \
    .groupBy('_c1', 'UniqueCarrier') \
    .count().orderBy('count', ascending=False).show()

+--------------------+-------------+-----+
|                 _c1|UniqueCarrier|count|
+--------------------+-------------+-----+
|  Southwest Airlines|           WN|54557|
|     Delta Air Lines|           DL|28962|
|   American Airlines|           AA|26291|
|     United Airlines|           UA|19594|
|             SkyWest|           OO|17924|
|Atlantic Southeas...|           EV|12340|
|     JetBlue Airways|           B6|10406|
|     Alaska Airlines|           AS| 4966|
|     Spirit Airlines|           NK| 4435|
|   Frontier Airlines|           F9| 3181|
|      Virgin America|           VX| 2871|
|   Hawaiian Airlines|           HA| 2265|
+--------------------+-------------+-----+



### Q23

Compute the number of delays per company per day. The day is encoded as an integer in the column `DayOfWeek` in `fight_info`. You can display the day as an integer or map it into a string name of the week.
Sort the data by airline code (UniqueCarrier) and by increasing values of DayOfWeek


You results should look like the following



In [453]:
flight_info_df.join(airlines_info, flight_info_df.UniqueCarrier == airlines_info._c3, 'inner') \
    .where(flight_info_df["DepTime"] > flight_info_df["CRSDepTime"]) \
    .groupBy('_c1', 'UniqueCarrier', 'DayOfWeek') \
    .count() \
    .orderBy('UniqueCarrier', 'DayOfWeek', ascending=True).show()


+-----------------+-------------+---------+-----+
|              _c1|UniqueCarrier|DayOfWeek|count|
+-----------------+-------------+---------+-----+
|American Airlines|           AA|        1| 5117|
|American Airlines|           AA|        2| 3688|
|American Airlines|           AA|        3| 2941|
|American Airlines|           AA|        4| 3575|
|American Airlines|           AA|        5| 3525|
|American Airlines|           AA|        6| 2800|
|American Airlines|           AA|        7| 4645|
|  Alaska Airlines|           AS|        1|  872|
|  Alaska Airlines|           AS|        2|  669|
|  Alaska Airlines|           AS|        3|  618|
|  Alaska Airlines|           AS|        4|  728|
|  Alaska Airlines|           AS|        5|  630|
|  Alaska Airlines|           AS|        6|  517|
|  Alaska Airlines|           AS|        7|  932|
|  JetBlue Airways|           B6|        1| 1892|
|  JetBlue Airways|           B6|        2| 1787|
|  JetBlue Airways|           B6|        3| 1253|


### Q24  

Counting the number of delayed flights per airline is misleading, as airlines with more flights are more likley to have delays than companies with substantially fiewer flights. 

Repeat the same query above but, for each carrier, normalize the counts of delays by the total number of flights for that carrier. 


In [481]:
import pyspark.sql.functions as F


joined_df = flight_info_df.join(airlines_info, flight_info_df.UniqueCarrier == airlines_info._c3, 'inner')

delays_df = joined_df \
    .where(flight_info_df["DepTime"] > flight_info_df["CRSDepTime"]) \
    .groupBy('_c1', 'UniqueCarrier', 'DayOfWeek') \
    .count()
    
total_delays_by_airline = delays_df \
    .groupBy('UniqueCarrier') \
    .sum()

normalized_df = delay_count_df \
                .join(total_delays_by_airline, ['UniqueCarrier']) \
                .withColumn("normalized_count", F.col('count') / F.col('sum(count)'))
normalized_df.orderBy('UniqueCarrier', 'DayOfWeek', ascending=True).show()

+-------------+-----------------+---------+-----+----------+-------------------+
|UniqueCarrier|              _c1|DayOfWeek|count|sum(count)|   normalized_count|
+-------------+-----------------+---------+-----+----------+-------------------+
|           AA|American Airlines|        1| 5117|     26291| 0.1946293408390704|
|           AA|American Airlines|        2| 3688|     26291| 0.1402761401239968|
|           AA|American Airlines|        3| 2941|     26291|0.11186337529953216|
|           AA|American Airlines|        4| 3575|     26291|0.13597809136206307|
|           AA|American Airlines|        5| 3525|     26291|0.13407629987448177|
|           AA|American Airlines|        6| 2800|     26291|0.10650032330455289|
|           AA|American Airlines|        7| 4645|     26291|0.17667642919630291|
|           AS|  Alaska Airlines|        1|  872|      4966|  0.175594039468385|
|           AS|  Alaska Airlines|        2|  669|      4966| 0.1347160692710431|
|           AS|  Alaska Airl

 ### Q25 

Time the query above. How long did it take to run. 
  * Make sure you run the code a few times and compute the average run time.
  * The above should be easy to implement if you use the correct Jupyter Notebook `magic` function
  

In [500]:
%%timeit

joined_df = flight_info_df.join(airlines_info, flight_info_df.UniqueCarrier == airlines_info._c3, 'inner')
delays_df = joined_df \
    .where(flight_info_df["DepTime"] > flight_info_df["CRSDepTime"]) \
    .groupBy('_c1', 'UniqueCarrier', 'DayOfWeek') \
    .count()

total_delays_by_airline = delays_df \
    .groupBy('UniqueCarrier') \
    .sum()

normalized_df = delay_count_df \
                .join(total_delays_by_airline, ['UniqueCarrier']) \
                .withColumn("normalized_count", F.col('count') / F.col('sum(count)'))
normalized_df.orderBy('UniqueCarrier', 'DayOfWeek', ascending=True).show()

+-------------+-----------------+---------+-----+----------+-------------------+
|UniqueCarrier|              _c1|DayOfWeek|count|sum(count)|   normalized_count|
+-------------+-----------------+---------+-----+----------+-------------------+
|           AA|American Airlines|        1| 5117|     26291| 0.1946293408390704|
|           AA|American Airlines|        2| 3688|     26291| 0.1402761401239968|
|           AA|American Airlines|        3| 2941|     26291|0.11186337529953216|
|           AA|American Airlines|        4| 3575|     26291|0.13597809136206307|
|           AA|American Airlines|        5| 3525|     26291|0.13407629987448177|
|           AA|American Airlines|        6| 2800|     26291|0.10650032330455289|
|           AA|American Airlines|        7| 4645|     26291|0.17667642919630291|
|           AS|  Alaska Airlines|        1|  872|      4966|  0.175594039468385|
|           AS|  Alaska Airlines|        2|  669|      4966| 0.1347160692710431|
|           AS|  Alaska Airl

+-------------+-----------------+---------+-----+----------+-------------------+
|UniqueCarrier|              _c1|DayOfWeek|count|sum(count)|   normalized_count|
+-------------+-----------------+---------+-----+----------+-------------------+
|           AA|American Airlines|        1| 5117|     26291| 0.1946293408390704|
|           AA|American Airlines|        2| 3688|     26291| 0.1402761401239968|
|           AA|American Airlines|        3| 2941|     26291|0.11186337529953216|
|           AA|American Airlines|        4| 3575|     26291|0.13597809136206307|
|           AA|American Airlines|        5| 3525|     26291|0.13407629987448177|
|           AA|American Airlines|        6| 2800|     26291|0.10650032330455289|
|           AA|American Airlines|        7| 4645|     26291|0.17667642919630291|
|           AS|  Alaska Airlines|        1|  872|      4966|  0.175594039468385|
|           AS|  Alaska Airlines|        2|  669|      4966| 0.1347160692710431|
|           AS|  Alaska Airl

### Q26 

Use one of the techniques covered in class to accelerate this query. Time your query to see by how much the run time was improved

In [503]:
%%timeit

joined_df = flight_info_df.join(airlines_info, flight_info_df.UniqueCarrier == airlines_info._c3, 'inner')
joined_df = joined_df.coalesce(1)

delays_df = joined_df \
    .where(flight_info_df["DepTime"] > flight_info_df["CRSDepTime"]) \
    .groupBy('_c1', 'UniqueCarrier', 'DayOfWeek') \
    .count()

total_delays_by_airline = delays_df \
    .groupBy('UniqueCarrier') \
    .sum()

normalized_df = delay_count_df \
                .join(total_delays_by_airline, ['UniqueCarrier']) \
                .withColumn("normalized_count", F.col('count') / F.col('sum(count)'))

normalized_df.orderBy('UniqueCarrier', 'DayOfWeek', ascending=True).show()

+-------------+-----------------+---------+-----+----------+-------------------+
|UniqueCarrier|              _c1|DayOfWeek|count|sum(count)|   normalized_count|
+-------------+-----------------+---------+-----+----------+-------------------+
|           AA|American Airlines|        1| 5117|     26291| 0.1946293408390704|
|           AA|American Airlines|        2| 3688|     26291| 0.1402761401239968|
|           AA|American Airlines|        3| 2941|     26291|0.11186337529953216|
|           AA|American Airlines|        4| 3575|     26291|0.13597809136206307|
|           AA|American Airlines|        5| 3525|     26291|0.13407629987448177|
|           AA|American Airlines|        6| 2800|     26291|0.10650032330455289|
|           AA|American Airlines|        7| 4645|     26291|0.17667642919630291|
|           AS|  Alaska Airlines|        1|  872|      4966|  0.175594039468385|
|           AS|  Alaska Airlines|        2|  669|      4966| 0.1347160692710431|
|           AS|  Alaska Airl

+-------------+-----------------+---------+-----+----------+-------------------+
|UniqueCarrier|              _c1|DayOfWeek|count|sum(count)|   normalized_count|
+-------------+-----------------+---------+-----+----------+-------------------+
|           AA|American Airlines|        1| 5117|     26291| 0.1946293408390704|
|           AA|American Airlines|        2| 3688|     26291| 0.1402761401239968|
|           AA|American Airlines|        3| 2941|     26291|0.11186337529953216|
|           AA|American Airlines|        4| 3575|     26291|0.13597809136206307|
|           AA|American Airlines|        5| 3525|     26291|0.13407629987448177|
|           AA|American Airlines|        6| 2800|     26291|0.10650032330455289|
|           AA|American Airlines|        7| 4645|     26291|0.17667642919630291|
|           AS|  Alaska Airlines|        1|  872|      4966|  0.175594039468385|
|           AS|  Alaska Airlines|        2|  669|      4966| 0.1347160692710431|
|           AS|  Alaska Airl

### Q27 

Is the departure delay (i.e., DepTime - CRSDepTime) predictive of the arrival delay (ArrTime > CRSArrTime)?
Use an approach of your choice (e.g. `skelearn` which we covered in class or `Spark`) to model as a linear regression the arrival delay as a function of the departure delay. 



In [651]:

from pyspark.ml.regression import LinearRegression

# Load all delays
delays_df = joined_df \
    .where(flight_info_df["DepTime"] > flight_info_df["CRSDepTime"])


# Compute departure and arrival delays
delays_df = delays_df.withColumn("dep_delay", F.col('DepTime') - F.col('CRSDepTime'))
delays_df = delays_df.withColumn("arr_delay", F.col('ArrTime') - F.col('CRSArrTime'))

delays_df.select(['DepTime', 'ArrTime']).show()
# Assemble feature vector
from pyspark.ml.feature import VectorAssembler
vA = VectorAssembler(inputCols = ['dep_delay'], outputCol = 'features')
features_df = vA.transform(delays_df)
features_df = features_df.select(['features', 'arr_delay'])
features_df = features_df.dropna()

# split training and test 80/20
training, testing = features_df.randomSplit([0.8, 0.2])

# zip lists together so we have a list of tuples with (departure delay, arrival delay)
lr = LinearRegression(featuresCol = 'features',
                      labelCol='arr_delay',
                      maxIter=15,
                      regParam=0.3,
                      elasticNetParam=0.8)

lr_model = lr.fit(training)

from pyspark.ml.evaluation import RegressionEvaluator
lr_pred = lr_model.transform(testing)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="arr_delay",metricName="rmse")

print("RMSE on training: %f" % lr_model.summary.rootMeanSquaredError)
print("RMSE on testing data = %f" % lr_evaluator.evaluate(lr_pred))


# The departure delay is not a good predictor for arrival delay

TypeError: strptime() argument 1 must be str, not Column